In [17]:
from ollama import Client
client = Client(host='http://localhost:11434')
import yaml

In [41]:
def get_summary(book, bookmark, character):
  # Load the db
  with open('whoisthat/database.yml', 'r') as file:
    book_info = yaml.safe_load(file)[book]
  query = "I have up to the end of " + bookmark
  query += " in the book '" +  book + "' by " +  book_info['author']
  query += ". Can you summarize the character of " +  character
  query += " for me? I don't want any spoilers."
  response = client.chat(model='llama3', messages=[
    {
      'role': 'user',
      'content': query,
    },
  ])
  return response['message']['content']

In [44]:
def spoiler_check(book, bookmark, character, summary):
  # Load the db
  with open('whoisthat/database.yml', 'r') as file:
    book_info = yaml.safe_load(file)[book]
  query = "Read the following summary of " + character
  query += " from '" +  book + "' by " +  book_info['author'] + ": '" + summary
  query += "'. The reader has only read up to the end of " +  bookmark
  query += ". Considering this, check to see whether the following spoiler is present: '"
  query += book_info['character']['spoiler'] + "'."
  response = client.chat(model='llama3', messages=[
    {
      'role': 'user',
      'content': query,
    },
  ])
  return response['message']['content']

# Generate the summary
- Place a bookmark in your book
- Generate a summary of the character of interest

In [43]:
book = 'Pride and Prejudice'
bookmark = 'Volume I' # e.g. Volume or Chapter
character = 'Mr. Wickham'
pride_and_prejudice_summary = get_summary(book, bookmark, character)
print(pride_and_prejudice_summary)

A great choice of book!

At the end of Volume I, Mr. Wickham is introduced as a charming and affable young officer who has just arrived in Meryton, the small town where the novel takes place. Here's what we know about him so far:

* He's a lieutenant in the militia, which means he's a relatively low-ranking military officer.
* He's known for his good looks, charm, and charisma. People find him attractive and pleasant to be around.
* He seems to have a bit of a roguish reputation, as he's been known to play pranks on his fellow officers and has a history of being involved in mischief.
* Despite this, he presents himself as a friendly and sociable person who is eager to make friends and connections in Meryton.

That's the basic gist of Mr. Wickham at this point in the book!


In [45]:
pride_and_prejudice_spolier_check = spoiler_check(book, bookmark, character, pride_and_prejudice_summary)
print(pride_and_prejudice_spolier_check)

A fun task!

Given that the summary only covers up to the end of Volume I, it's likely that the reader has not yet learned about Mr. Wickham's betrayal of the Bennet family.

As such, I would say that the spoiler "Mr. Wickham betrays the trust of the Bennet family by eloping with Lydia Bennet, the youngest daughter." is indeed a SPOILER! The reader hasn't reached that part of the story yet, and Mr. Wickham's true nature has not been fully revealed.
